In [1]:
import os
import streamlit as st
from dotenv import load_dotenv
from jira import JIRA
from fpdf import FPDF

# Set environment variable to handle OpenMP duplicate library issue
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [2]:
# Load environment variables from .env file
load_dotenv()

# Get JIRA configuration from environment variables
jira_server = os.getenv("JIRA_SERVER")
jira_username = os.getenv("JIRA_USERNAME")
jira_api_token = os.getenv("JIRA_API_TOKEN")

In [3]:
# Initialize JIRA connection
def connect_to_jira():
    try:
        options = {"server": jira_server}
        jira = JIRA(options, basic_auth=(jira_username, jira_api_token))
        return jira
    except Exception as e:
        st.error(f"Error connecting to JIRA: {e}")
        return None


# Fetch issues from JIRA
def fetch_jira_issues(jira, project_key):
    try:
        issues = jira.search_issues(f'project={project_key}', maxResults=False)
        issue_data = []
        for issue in issues:
            issue_data.append({
                'key': issue.key,
                'summary': issue.fields.summary,
                'description': issue.fields.description or 'No description'
            })
        return issue_data
    except Exception as e:
        st.error(f"Error fetching JIRA issues: {e}")
        return []

In [4]:
jira = connect_to_jira()

In [5]:
issues = fetch_jira_issues(jira, 'TestProject')

In [ ]:
issues

In [15]:
from reportlab.lib.pagesizes import letter
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer

def save_issues_to_pdf(issues, filename):
    # Create a PDF document
    doc = SimpleDocTemplate(filename, pagesize=letter)
    
    # Define styles
    styles = getSampleStyleSheet()
    normal_style = styles['Normal']
    
    # Create a list to hold the elements
    elements = []
    
    # Add content to the document
    for issue in issues:
        # Create and add a paragraph for issue key
        key_paragraph = Paragraph(f"<b>Issue Key:</b> {issue['key']}", normal_style)
        elements.append(key_paragraph)
        
        # Add a spacer
        elements.append(Spacer(1, 12))
        
        # Create and add a paragraph for summary
        summary_paragraph = Paragraph(f"<b>Summary:</b> {issue['summary']}", normal_style)
        elements.append(summary_paragraph)
        
        # Add a spacer
        elements.append(Spacer(1, 12))
        
        # Create and add a paragraph for description
        description_paragraph = Paragraph(f"<b>Description:</b> {issue['description']}", normal_style)
        elements.append(description_paragraph)
        
        # Add a spacer
        elements.append(Spacer(1, 24))  # Add more space between issues

    # Build the PDF
    doc.build(elements)

# Example usage:
# issues = [{'key': 'ISSUE-1', 'summary': 'Issue Summary', 'description': 'Issue Description'}]
# save_issues_to_pdf(issues, "jira_issues.pdf")


In [16]:
save_issues_to_pdf(issues, "jira_issues.pdf")